<a href="https://colab.research.google.com/github/RodrigoLSobrinho/Personal/blob/main/C%C3%B3pia_de_C%C3%A1lculo_de_TR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("Simulacao_maturacao_1.csv")

In [ ]:
def filter_tr_values(row):
    try:
        if row['Taxa de Transformação [v/v]'] > 1:
            row = 1
        if row['Taxa de Transformação [v/v]'] < 0:
            row = 0
        else:
            row = row['Taxa de Transformação [v/v]']
        return row
    except:
        row = 1
        return row

In [ ]:
def tr_lamina_dagua(simulations_df, mediana_minimo, lamina_dagua, variavel_hi, variavel_hia, variavel_profundidade):
    # O cálculo de tr seguirá os seguintes passos:
    # 1. Calcular o tr em todas as amostras do poço com a equaçao Tr = (HIa - HI)/HIa onde HIa é indice de hidrogenio ativo e o HI é o indice de hidrogenio
    # 2. Calcular a mediana ou o mínimo de todos os valores de Tr calculados no passo 1.
    # 3. Onde a profunddidade for igual ou menor que a laminadagua, o valor de Tr será igual a zero.
    # 4. Ajustar uma curva logaritmica com a profundidade da lamina dagua e a profundidade final e o tr na lamina dagua (zero) e o tr na profundidade final (mediana ou mínimo), utilizando scipy curve_fit
    import scipy

    # 1. Calcular o tr em todas as amostras do poço com a equaçao Tr = (HIa - HI)/HIa onde HIa é indice de hidrogenio ativo e o HI é o indice de hidrogenio
    simulations_df['Tr'] = (simulations_df[variavel_hia] - simulations_df[variavel_hi]) / simulations_df[variavel_hia]
    simulations_df["Tr"] = df.apply(filter_tr_values, axis=1)

    # 2. Calcular a mediana ou o mínimo de todos os valores de Tr calculados no passo 1.
    if mediana_minimo == 'mediana':
        tr_mediana = simulations_df['Tr'].median()
    if mediana_minimo == 'minimo':
        tr_mediana = simulations_df['Tr'].min()

    # 3. Onde a profunddidade for igual ou menor que a laminadagua, o valor de Tr será igual a zero.
    simulations_df.loc[simulations_df[variavel_profundidade] <= lamina_dagua, 'Tr'] = 0

    print(simulations_df["Tr"])
    print(tr_mediana)
    # 4. Ajustar uma curva logaritmica com a profundidade da lamina dagua e a profundidade final e o tr na lamina dagua (zero) e o tr na profundidade final (mediana ou mínimo), utilizando scipy curve_fit
    x = [lamina_dagua, simulations_df[variavel_profundidade].max()]
    y = [0, tr_mediana]

    log_fit = scipy.optimize.curve_fit(lambda t, a, b: a + b * np.log(t), x, y)
    a = log_fit[0][0]
    b = log_fit[0][1]

    # 5. Calcular o tr em todas as amostras do poço com a equaçao Tr = a + b * ln(profundidade)
    simulations_df['Tr'] = a + b * np.log(simulations_df[variavel_profundidade])

    # Extra: aplicando novamente filtro para que tr seja 0 abaixo da lamina d'agua
    simulations_df.loc[simulations_df[variavel_profundidade] <= lamina_dagua, 'Tr'] = 0

    return simulations_df

Dizendo para a função quais sao os nomes das variáveis correspondentes no df:

In [ ]:
variavel_hia = 'IH₀ Ativo [mg HC/g COT]'
variavel_hi = 'IH [mg HC/g COT'
variavel_profundidade = "Profundidade [m]"
mediana_minimo = "mediana"
lamina_dagua = 3000

In [ ]:
df = tr_lamina_dagua(df, "minimo", lamina_dagua, variavel_hi, variavel_hia, variavel_profundidade)

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
123    0.466024
124    0.468820
125    0.471619
126    0.474421
127    0.480030
Name: Tr, Length: 128, dtype: float64
0.0


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning:

Covariance of the parameters could not be estimated



In [ ]:
import plotly.express as px

In [ ]:
px.line(y='Tr', x=variavel_profundidade, data_frame=df)